In [ ]:
from ondepi.applications import price_level as pl
from ondepi.visualisations import sample as plot_sample
from ondepi.resources import utils
import pandas as pd

In [ ]:
symbol = 'INTC'
date = '2019-01-31'
price_levels = list(range(465000, 466100, 100))
direction = 1
std_size = 500
t0 = 1.05e3
t1 = 1.50e3
reset_time_origin = True

In [ ]:
lob = []
for price_level in price_levels:
    queue = pl.init_queue_on_lobster_data(
        symbol=symbol, date=date,
        price_level=price_level, direction=direction, std_size=std_size,
        t0=t0, t1=t1, reset_time_origin=reset_time_origin
    )
    queue = pl.set_param(queue)
    lob.append(queue)

In [ ]:
for queue in lob:
    queue.simulate(max_time=1.0e3, max_events=1000, first_event=1, first_state=5)
    queue.filter(dt=7.5e-3, num_states=100)

In [ ]:
dfs = []
for queue, price_level in zip(lob, price_levels):
    dfs.append(
        pl.produce_df_detection(queue, beta=150.5, price_level=price_level, event_times_only=False))

In [ ]:
df_all = dfs[0].copy().set_index((price_levels[0], 'idx'))
for df, price_level in zip(dfs[1:], price_levels[1:]):
    df = df.set_index((price_level, 'idx'))
    df_all = df_all.merge(df, left_index=True, right_index=True, how='outer')
df_all.sort_index(axis=0, inplace=True)
df_all.ffill(inplace=True)

In [ ]:
df_sum = df_all[price_levels[0]].loc[:, ['state', 'predictor', 'error', 'detector']].copy()
for price_level in price_levels[1:]:
    df_sum += df_all[price_level].loc[:, ['state', 'predictor', 'error', 'detector']]
df_sum = df_sum.merge(df_all[price_levels[0]].loc[:, ['time sample', 'time filter']], 
                      left_index=True, right_index=True)

In [ ]:
plot_sample.plot_impact(df_sum, time_range=(5, 500), title='Price impact', std_size=std_size, label='Price impact')